## apriori algorithm


### import libraries

In [283]:
import numpy as np
import pandas as pd
import itertools 
from sklearn.preprocessing import KBinsDiscretizer
from scipy.sparse import csr_matrix

### load data
1. load data from csv file
2. convert data to binary data
3. convert data to list of lists

In [284]:
def load_data(path):
    
    data = pd.read_csv(path)

    return data

def process(data):

    # convert data to numpy array
    data = data.to_numpy()

    return data

# def process_noneBianry(data):

#     # Split nominal and numerical data
#     nominal_data = data.select_dtypes(include=['object'])
#     numerical_data = data.select_dtypes(include=['int', 'float'])
    
#     # One-hot encode nominal data
#     nominal_data = pd.get_dummies(nominal_data)

#     # Discretize numerical data
#     discretizer = KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform')
#     discretized_data = pd.DataFrame(discretizer.fit_transform(numerical_data), columns=numerical_data.columns)

#     # Concatenate nominal and numerical data
#     processed_data = pd.concat([nominal_data, discretized_data], axis=1)

#     # Convert data to binary format
#     processed_data = np.where(processed_data > 0, 1, 0)

#     # Save processed data to file
#     np.savetxt("processed_data.csv", processed_data, delimiter=",", fmt='%d')

#     return processed_data


def data_to_transactions(data):
    
    transactions = []
    for row in data:
        row = np.where(row > 0)[0]
        transactions.append(row)

    return transactions

In [285]:
data = load_data("ex.csv")

# data = load_data("basket_analysis.csv")

variables = data.columns

data = process(data)

data = data_to_transactions(data)

### apriori algorithm

1. generate candidate itemsets
2. generate frequent itemsets
3. generate association rules

In [286]:
def generate_candidate_itemsets(frequent_itemsets, k):
    
    if k == 2:
        candidate_itemsets = []
        for i in range(len(frequent_itemsets)):
            for j in range(i+1, len(frequent_itemsets)):
                itemset_1 = [frequent_itemsets[i]]
                # print (itemset_1)
                itemset_2 = [frequent_itemsets[j]]
                # print (itemset_2)
                if itemset_1[:k-2] == itemset_2[:k-2]:
                    candidate_itemset = itemset_1 + [itemset_2[-1]]
                    candidate_itemset.sort()
                    candidate_itemsets.append(set(candidate_itemset))
                
    else:
        candidate_itemsets = []
        for i in range(len(frequent_itemsets)):
            for j in range(i+1, len(frequent_itemsets)):
                itemset_1 = frequent_itemsets[i]
                itemset_2 = frequent_itemsets[j]
                candidate_itemset = itemset_1.union(itemset_2)
                if len(candidate_itemset) == k and candidate_itemset not in candidate_itemsets:
                    candidate_itemsets.append(candidate_itemset)

    return candidate_itemsets

def get_frequency(transactions):
    item_counts = {}
    for transaction in transactions:
        for item in transaction:
            if item in item_counts:
                item_counts[item] += 1
            else:
                item_counts[item] = 1
    
    return item_counts


def closed(itemsets, frequency):
    closed_itemsets = []
    leng = 0
    for itemset in range(len(itemsets)):
        itemsetlen = len(itemsets[itemset])
        print(itemsetlen)
        leng += itemsetlen
        print(leng)
        

        for item in range (len(itemsets[itemset])):
            
            equ = True
            # print(itemsets[itemset][item])

            if itemset+1 < len(itemsets):
                # print("item",item)    
                for supersets in range(len(itemsets[itemset+1])):
                    # print("super",supersets)  
                    # print(itemsets[itemset+1][supersets])
                    if isinstance(itemsets[itemset][item], set):
                        if itemsets[itemset][item].issubset(itemsets[itemset+1][supersets]):
                            
                            if frequency[item] == frequency[leng+supersets] :
                                if itemsets[itemset][item] == {0,2}:
                                    print("item",item) 
                                    
                                equ = False
                                
                                break

                    else:
                        if itemsets[itemset][item] in itemsets[itemset+1][supersets]:
  
                            if frequency[item] == frequency[leng+supersets] :
                             
                                equ = False
                                break
                            
                if equ == True and itemsets[itemset][item] not in closed_itemsets:
                    closed_itemsets.append(itemsets[itemset][item])

    return closed_itemsets


def association_rules(itemsets):
    association_rules = []
    for itemset in itemsets:
        for i in range(len(itemset)):
            # generate all possible left-hand side (LHS) of the association rule
            for lhs in itertools.combinations(itemset, i):
                lhs = set(lhs)
                # generate the right-hand side (RHS) of the association rule
                rhs = itemset - lhs
                if lhs and rhs:
                    association_rules.append((lhs, rhs))
    return association_rules


def confidence(itemsets, transactions, min_confidence):
    rules = association_rules(itemsets)
    confidence_rules = []
    for rule in rules:
        lhs = rule[0]
        rhs = rule[1]
        lhs_count = 0
        rhs_count = 0
        for transaction in transactions:
            if lhs.issubset(transaction):
                lhs_count += 1
                if rhs.issubset(transaction):
                    rhs_count += 1
        confidence = rhs_count / lhs_count
        if confidence >= min_confidence:
            confidence_rules.append((lhs, rhs, confidence))
    return confidence_rules


def flatten_list(lst):
    flattened = []
    for element in lst:
        if isinstance(element, list):
            flattened.extend(flatten_list(element))
        else:
            flattened.append(element)
    return flattened
        

In [287]:
def apriori(transactions, min_support):
    # Counting the frequency of each item in the dataset
    item_counts = get_frequency(transactions)

    # Removing infrequent items from the item_counts dictionary
    item_counts = {k: v for k, v in item_counts.items() if v >= min_support}

    # Extracting frequent items and their counts
    frequent_items = list(item_counts.keys())
    frequent_items_counts = list(item_counts.values())

    
    # Finding frequent itemsets
    frequent_itemsets = [frequent_items]
    # print (frequent_itemsets) 

    k = 2
    while True:
        # Generating candidate itemsets
        
        # print(frequent_itemsets[-1])
        candidate_itemsets = generate_candidate_itemsets(frequent_itemsets[-1], k)
        # print(candidate_itemsets)

        # Counting the frequency of each candidate itemset in the dataset
        candidate_itemsets_counts = [0] * len(candidate_itemsets)
        for transaction in transactions:
            for i, itemset in enumerate(candidate_itemsets):
                if set(itemset).issubset(set(transaction)):
                    candidate_itemsets_counts[i] += 1
        # print(candidate_itemsets_counts)

        # Removing infrequent candidate itemsets
        frequent_itemsets_k = []
        frequent_itemsets_counts_k = []
        for i, itemset in enumerate(candidate_itemsets):
            if candidate_itemsets_counts[i] >= min_support:
                frequent_itemsets_k.append(itemset)
                frequent_itemsets_counts_k.append(candidate_itemsets_counts[i])
        
        # print(frequent_itemsets_k)
        # Stopping if no frequent itemsets were found
        if frequent_itemsets_k == []:
            break
        
        # Adding frequent itemsets to the list
        frequent_itemsets.append(frequent_itemsets_k)
        frequent_items_counts.append(frequent_itemsets_counts_k)
        # print (frequent_itemsets)

        k += 1

    
    
    # Flattening the list of frequent itemsets and their counts
    frequent_items_counts = flatten_list(frequent_items_counts)

    close = closed(frequent_itemsets, frequent_items_counts)

    rules = association_rules(flatten_list(frequent_itemsets[1:]))

    print("rules", rules)

    # confi = confidence(flatten_list(frequent_itemsets[1:]), transactions, min_confidence)

   
    return frequent_itemsets, frequent_items_counts, close, rules
  


In [288]:
min_support = 3
# min_confidence = 1

frequent_itemsets, frequent_itemsets_counts, closed, rules = apriori(data, min_support)

print("Frequent itemsets:", frequent_itemsets)
print("Frequent itemsets counts:", frequent_itemsets_counts)
print("Closed itemsets:", closed)

# print (association_rule(frequent_itemsets))


5
5
4
9
1
10
rules [({0}, {1}), ({1}, {0}), ({0}, {2}), ({2}, {0}), ({1}, {2}), ({2}, {1}), ({2}, {5}), ({5}, {2}), ({0}, {1, 2}), ({1}, {0, 2}), ({2}, {0, 1}), ({0, 1}, {2}), ({0, 2}, {1}), ({1, 2}, {0})]
Frequent itemsets: [[0, 1, 2, 3, 5], [{0, 1}, {0, 2}, {1, 2}, {2, 5}], [{0, 1, 2}]]
Frequent itemsets counts: [4, 4, 4, 3, 3, 4, 3, 3, 3, 3]
Closed itemsets: [2, 3, {0, 1}, {0, 2}, {1, 2}, {2, 5}]
